In [ ]:
# libs: splinter, bs4, requests

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser

# Gel list of all cases and solve rates

In [ ]:
def parse_case_links(links):
    urls = []
    names = []
    for link in links:
        url = link.get("href")
        if url.startswith("/mystery/"):
            urls.append(url)
            names.append(link.text)
    return urls, names

In [ ]:
def parse_author_links(links):
    urls = []
    names = []
    for link in links:
        url = link.get("href")
        if url.startswith("/author/"):
            urls.append(url)
            names.append(link.text)
    return urls, names

In [ ]:
num_pages = 48

In [ ]:
all_case_urls = []
all_case_names = []
all_author_urls = []
all_author_names = []
all_attempts = []
all_solve_rates = []


for pn in range(1, num_pages+1):
    print("Page number: ", pn)
    URL = f"https://www.5minutemystery.com/archives?page={pn}&type=&keywords="
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    table = soup.find(lambda tag: tag.name=='table')
    all_hyperlinks = table.find_all("a")
    case_urls, case_names = parse_case_links(all_hyperlinks)
    author_urls, author_names = parse_author_links(all_hyperlinks)
    attempts = table.findAll("td", class_="num hidden-phone")
    solve_rates = table.findAll("td", class_="num")

    all_case_urls.extend(case_urls)
    all_case_names.extend(case_names)
    all_author_urls.extend(author_urls)
    all_author_names.extend(author_names)
    all_attempts.extend(attempts)
    all_solve_rates.extend(solve_rates)

In [ ]:
# fix solve rates
# we only need to take every other solve rate
# because the first solve rate is the number of attempts
# and the second solve rate is the percentage of solve rate
all_solve_rates = all_solve_rates[1::2]

In [ ]:
# get text from attemts and solve rates
all_attempts = [attempt.text for attempt in all_attempts]
all_solve_rates = [solve_rate.text for solve_rate in all_solve_rates]

In [ ]:
# parse comma seprated numbers in attempts to ints
all_attempts = [int(attempt.replace(",", "")) for attempt in all_attempts]

# parse percentage to floats in solve rates
all_solve_rates = [float(solve_rate.replace("%", "")) for solve_rate in all_solve_rates]

In [ ]:
# save lsits to csv

df = pd.DataFrame({"case_url": all_case_urls, "case_name": all_case_names, "author_url": all_author_urls, "author_name": all_author_names, "attempts": all_attempts, "solve_rate": all_solve_rates})
# drop duplicates where case_url are the same
df = df.drop_duplicates(subset="case_url")

In [6]:
# add https://www.5minutemystery.com to each case_url and author_url in dataframe
df['case_url'] = 'https://www.5minutemystery.com' + df['case_url']
df['author_url'] = 'https://www.5minutemystery.com' + df['author_url']

In [7]:
df.to_csv("links.csv", index=False)

In [8]:
# read links.csv
df = pd.read_csv("links.csv")

In [9]:
len(df)

191

In [10]:
df.head()

,case_url,case_name,author_url,author_name,attempts,solve_rate
0,https://www.5minutemystery.com/mystery/sweat-i...,Sweat it Out,https://www.5minutemystery.com/author/mysteryman,Nick Andreychuk,1200,39.4
1,https://www.5minutemystery.com/mystery/mystery...,Mystery of the Missing Heart,https://www.5minutemystery.com/author/mike_wever,Mike Wever,3274,65.1
2,https://www.5minutemystery.com/mystery/stealin...,Stealing Second Base,https://www.5minutemystery.com/author/BillShepard,William Shepard,1452,57.0
3,https://www.5minutemystery.com/mystery/murder-...,Murder in the Old House,https://www.5minutemystery.com/author/tfowler,Tom Fowler,4056,54.7
4,https://www.5minutemystery.com/mystery/the-che...,The Chess Mystery,https://www.5minutemystery.com/author/mzilla,Moe Zilla,2104,50.0


# Get puzzle texts and suspects lists

In [11]:
df = pd.read_csv("links.csv")

In [13]:
browser = Browser('edge', headless=False)

In [14]:
browser.visit("https://www.5minutemystery.com")
# Before proceedding sign in now manually with your login and password

In [ ]:
suspects_from_dropdawn = True 
all_mystery_texts = []
all_suspect_names = []
for i, case_url in enumerate(df['case_url']):
    full_url = f"https://www.5minutemystery.com{case_url}"
    browser.visit(full_url)
    browser.find_by_text('Attempt this mystery').click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    if not suspects_from_dropdawn:
        # get suspect names from the div with class section suspects
        suspects = soup.find('div', class_='section suspects').find_all('span')[1:]
        # get suspect names
        suspect_names = []
        for suspect in suspects:
            suspect_names.append(suspect.text)
    else:
        browser.links.find_by_partial_text("Choose one").click()
        suspect_names = browser.find_by_css('ul.dropdown-menu li.suspect')
        suspect_names = [s.text for s in suspect_names]

    all_suspect_names.append("; ".join(suspect_names))

    # get mistery text
    mystery_text = soup.find('div', id='mystery-full').text.strip()
    all_mystery_texts.append(mystery_text)

    if i % 20 == 0:
        print(f"Done with {i} cases")

browser.quit()

In [ ]:
# add suspect names and mistery texts to links dataframe
df['answer_options'] = all_suspect_names
df['mystery_text'] = all_mystery_texts

In [40]:
# save as a new csv file
df.to_csv('links_with_text.csv', index=False)

# Fetch answers for each puzzle

In [3]:
df = pd.read_csv('links_with_text.csv')

In [ ]:
browser = Browser('edge', headless=False)

In [ ]:
browser.visit("https://www.5minutemystery.com")
# Before proceedding sign in now manually with your login and password

In [ ]:
# iterate over all raws in full dataframe to get a suspect_names and case_url

all_guilty_suspects = []
for index, row in df.iterrows():

    # # skip non nan rows
    if row["guilty_suspect"] != "NAN":
        continue

    # get case_url
    case_url = row['case_url']

    # get suspect_names as a list
    suspect_names = row['answer_options']
    suspect_names = suspect_names.split("; ")

    ###
    guilty = "NAN"
    for suspect_name in suspect_names:
        # visit case_url
        browser.visit(case_url)
        browser.find_by_text('Attempt this mystery').click()
        browser.links.find_by_partial_text("Choose one").click()

        try:
            browser.links.find_by_partial_text(suspect_name).click()
            browser.find_by_value("Solve this mystery!").click()
        except:
            print("Could not find suspect_name: ", suspect_name, index)
            continue

        if len(browser.find_by_text("correct")) > 0:
            guilty = suspect_name
            break
    all_guilty_suspects.append(guilty)

    # print index every 10 rows
    if index % 10 == 0:
        print(index)


In [54]:
df['answer'] = all_guilty_suspects
df.at[50, "answer"] = "Washington, DC" # instead of just Washington

In [ ]:
# iterate over all mysteries
for i, row in df.iterrows():
    # split answer options by ";"
    suspects = row["answer_options"].split("; ")
    correct_suspect = row["answer"]
    correct_suspect_idx = suspects.index(correct_suspect)

    # df["answer_options"] is a ";"-separated string of answer options
    # the code below adds a number to each option
    # e.g. "Alang Edd; Max Crow; Sindy Elon" -> "(a) Alang Edd; (b) Max Crow; (c) Sindy Elon"
    # then it adds correct number to the df['answer'] column which is the correct answer
    # e.g. "Max Crow" -> "(b) Max Crow"
    suspects = [f"({chr(ord('a') + i)}) {s}" for i, s in enumerate(suspects)]
    correct_suspect = f"({chr(ord('a') + correct_suspect_idx)}) {correct_suspect}"

    suspects = "; ".join(suspects)
    # assign
    df.at[i, "answer_options"] = suspects
    df.at[i, "answer"] = correct_suspect

In [8]:
# save as a new csv file
df.to_csv('detective-puzzles.csv', index=False)

# Add chain-of-thought answers

In [9]:
# init edge browser
browser = Browser('edge', headless=False)

In [10]:
browser.visit("https://www.5minutemystery.com")
# Before proceedding sign in now manually with your login and password

In [19]:
full_answers = []
# iterate over puzzle links
for index, row in df.iterrows():
    # get puzzle link
    puzzle_link = row['case_url'] + "/results"
    # visit puzzle link
    browser.visit(puzzle_link)

    # click on button with text "Click here to read the full solution..."
    browser.find_by_text("Click here to read the full solution...").click()
    # get full answer from div with class "section solution-text"
    full_answer = browser.find_by_css('div.section.solution-text').text
    full_answers.append(full_answer)

In [144]:
browser.quit()

In [24]:
# replace double end of line symbols with a single in full answers
full_answers = [full_answer.replace("\n\n", "\n") for full_answer in full_answers]
# add full answers to dataframe
df['outcome'] = full_answers

In [ ]:
# import ast
# # read full.csv
# df_full = pd.read_csv('full.csv')
# df = pd.read_csv('detective-puzzles.csv')

# for index, row in df_full.iterrows():
#     suspect_names = row['answer_options']
#     df.at[index, 'answer_options'] = "; ".join(ast.literal_eval(suspect_names))

In [127]:
# change order of columns
df = df[['case_name', 'case_url', 'author_name', 'author_url', 'attempts', 'solve_rate', 'mystery_text', 'answer_options', 'answer', 'outcome']]

In [132]:
# strip each option in answer_options, full_answer, mystery_text
# full_answer and mystery_text are just strings
df['answer_options'] = df['answer_options'].apply(lambda x: "; ".join([s.strip() for s in x.split("; ")]))
df['outcome'] = df['outcome'].apply(lambda x: x.strip())
df['mystery_text'] = df['mystery_text'].apply(lambda x: x.strip())

In [135]:
# save as a new csv file
df.to_csv('detective-puzzles.csv', index=False)

In [143]:
df['author_name'].value_counts()

Moe Zilla            43
Tom Fowler           42
William Shepard      24
Laird Long           18
Robbie Cutler        12
Barney Parmington    10
Stefanina Hill        6
Steve Shrott          6
Nick Andreychuk       5
Nicholas LeVack       4
Ernest Capraro        2
Andrea Hein           2
Doug Fellin           2
Tammy-Lee Miller      2
Meghan Ford           1
Brad Marsh            1
Susanne Shaphren      1
Randy Godwin          1
Ryan Hogan            1
Matthew Lieff         1
Perry McCarney        1
Nicholas Lovell       1
Mike Wever            1
Meg A.  Write         1
Elsa Darcy            1
PIP Writer            1
Julie Hockenberry     1
Name: author_name, dtype: int64

In [141]:
# print all authors and hom many puzzles they have
df['author_name'].unique()
df['author_name'].value_counts()

# check how many stories in total wrote top 10 authors
df['author_name'].value_counts().head(10).sum()

170

In [142]:
df

,case_name,case_url,author_name,author_url,attempts,solve_rate,mistery_text,answer_options,answer,full_answer
0,Sweat it Out,https://www.5minutemystery.com/mystery/sweat-i...,Nick Andreychuk,https://www.5minutemystery.com/author/mysteryman,1200,39.4,Rubbernecking is a dangerous sport. I should k...,Chris Henderson; Dave Perkins; Larry Douglas; ...,Chris Henderson,"“Well, out with it!” Nathan exclaimed. “Or I’l..."
1,Mystery of the Missing Heart,https://www.5minutemystery.com/mystery/mystery...,Mike Wever,https://www.5minutemystery.com/author/mike_wever,3274,65.1,I was helping to clean up after the school pla...,Eric Winter; Jenny Jackson; Jimmy Jackson; Wen...,Eric Winter,"“Eric, you’ve got to return that heart to Mrs...."
2,Stealing Second Base,https://www.5minutemystery.com/mystery/stealin...,William Shepard,https://www.5minutemystery.com/author/BillShepard,1452,57.0,The Westbrook High School gymnasium was decora...,Coach Joe Morgan; Mary Thornton; Randy Newsom;...,Mary Thornton,I saw Principal Carol Jackson going into the f...
3,Murder in the Old House,https://www.5minutemystery.com/mystery/murder-...,Tom Fowler,https://www.5minutemystery.com/author/tfowler,4056,54.7,Todd Jensen recently inherited the old Jensen ...,"Bathroom; Bedroom of daughter, Anita Jensen; B...",Bathroom,Charlene looked into her empty cup and began t...
4,The Chess Mystery,https://www.5minutemystery.com/mystery/the-che...,Moe Zilla,https://www.5minutemystery.com/author/mzilla,2104,50.0,It was almost magic. All the chess pieces look...,Father; Greg; Tina; Uncle Larry,Greg,"""Did Dad steal the pieces?"" Tina asked.\n""He c..."
...,...,...,...,...,...,...,...,...,...,...
186,A Stolen Future,https://www.5minutemystery.com/mystery/a-stole...,Doug Fellin,https://www.5minutemystery.com/author/Dfellin,1692,61.1,George Wilson slid his access card through the...,Donna Blake; George Wilson; Jeffery Sharp; Pet...,Jeffery Sharp,"Before going into the conference room, I asked..."
187,The Dirty Half Dozen,https://www.5minutemystery.com/mystery/the-dir...,Tom Fowler,https://www.5minutemystery.com/author/tfowler,1137,37.5,The “Dirty Half Dozen” was a club of six recen...,Bethany Knight; Joe Clark; Sherry Fogle; Tonya...,Wayne Clark,"“Wayne, it had to be you.”\n“What! Why?”\n“Wel..."
188,A Porsche of Course,https://www.5minutemystery.com/mystery/a-porsc...,Randy Godwin,https://www.5minutemystery.com/author/Rgodwin,1265,36.8,When Martin Caldwell got to his office on Mond...,Amy Golden; Frankie Cole; Jeremy Steele; Lione...,Frankie Cole,"When Bill asked who it was, Martin explained t..."
189,The Mystery of the Missing Story,https://www.5minutemystery.com/mystery/the-mys...,Julie Hockenberry,https://www.5minutemystery.com/author/juliehoc...,1253,55.8,"“It snows and everyone becomes a kid again,” J...",Alex Rebmevon; Amy; Lucy; Sarah,Lucy,"“It must have been Alex,” Jack said, his cheek..."
